<a href="https://colab.research.google.com/github/dieko95/AlgoTrading/blob/develop/AlgoTrading.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AlgoTrading

## Download Alt Data (Tweets)

https://pypi.org/project/GetOldTweets3/



```
python Exporter.py --query @RedHat --since 2014-01-01 --until 2019-01-15 --maxtweets 5000000
```



In [1]:
# !pip install GetOldTweets3
!pip install langdetect

In [0]:
import pandas as pd
from pandas import Series, DataFrame
import numpy as np 
from langdetect import detect


## Sentiment Analysis with Vader - Nasty Prototype

![alt text](https://cdn-images-1.medium.com/max/800/1*G8yV2iaqqfaGfmRPRem2Fw.png)

In [3]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving redhatprototype.csv to redhatprototype (2).csv
User uploaded file "redhatprototype.csv" with length 8252 bytes


In [0]:
tweetsDf = pd.read_csv('redhatprototype.csv')

In [0]:
# !pip install vaderSentiment

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer


analyzer = SentimentIntensityAnalyzer()

tweet = []
number_favourites = []
retweets = []
vs_compound = []
vs_pos = []
vs_neu = []
vs_neg = []
timeStamp = []

for i in range(tweetsDf.shape[0]):
    tweet.append(tweetsDf.loc[i]['text'])
    number_favourites.append(tweetsDf.loc[i]['favorites'])
    retweets.append(tweetsDf.loc[i]['retweets'])
    timeStamp.append(tweetsDf.loc[i]['date'])
    vs_compound.append(analyzer.polarity_scores(tweetsDf.loc[i]['text'])['compound'])
    vs_pos.append(analyzer.polarity_scores(tweetsDf.loc[i]['text'])['pos'])
    vs_neu.append(analyzer.polarity_scores(tweetsDf.loc[i]['text'])['neu'])
    vs_neg.append(analyzer.polarity_scores(tweetsDf.loc[i]['text'])['neg'])

In [6]:

twitterDf = DataFrame({'Tweet': tweet,
                        'Favourites': number_favourites,
                        'retweets' : retweets,
                        'timeStamp' : timeStamp,
                        'Compound': vs_compound,
                        'Positive': vs_pos,
                        'Neutral': vs_neu,
                        'Negative': vs_neg})
twitterDf = twitterDf[['timeStamp','Tweet', 'Favourites', 'retweets','Compound',
                         'Positive', 'Neutral', 'Negative']]

twitterDf

,timeStamp,Tweet,Favourites,retweets,Compound,Positive,Neutral,Negative
0,2019-01-14 18:24,Tomorrow I am teaching a course on Buildah wit...,29,16,0.0000,0.000,1.000,0.000
1,2019-01-14 18:23,"Rob Szumski, Principal Product Manager at Red ...",3,2,-0.4939,0.041,0.853,0.106
2,2019-01-14 18:00,Wondering how 3rd party apps can be managed at...,0,0,0.6369,0.157,0.843,0.000
3,2019-01-14 17:27,Easily check real-time status of the #RedHat C...,4,4,0.6486,0.209,0.791,0.000
4,2019-01-13 12:42,In my two teams that I'm a part of at @redhat ...,7,0,0.5994,0.162,0.782,0.057
5,2019-01-13 11:50,@RedHat Cert Prep #RHCSA 3: Konfiguration von ...,2,2,0.0000,0.000,1.000,0.000
6,2019-01-13 11:50,I’ll be in Westford and working on the custome...,1,0,0.0000,0.000,1.000,0.000
7,2019-01-13 11:50,"@RedHat Cert Prep RHCSA 5: Bereitstellung, Kon...",2,2,0.0000,0.000,1.000,0.000
8,2019-01-13 11:50,@RedHat Cert Prep RHCSA 6: Verwaltung von Benu...,2,2,0.0000,0.000,1.000,0.000
9,2019-01-13 11:49,@RedHat Cert Prep RHCSA 7: Sicherheitsmanageme...,2,2,0.0000,0.000,1.000,0.000


In [0]:

twitterDf.Tweet = twitterDf.Tweet.apply(lambda x: x.lower()) #Lowers caps 
twitterDf.Tweet = twitterDf.Tweet.apply(lambda x: len(x.split()) < 7 and np.nan or x) #If the tweet is less than 7 characters create a NA place Holder 

twitterDf.Tweet = twitterDf.Tweet.replace(r'@\w*', '', regex = True) # Erases all usernames (word preceding @)
twitterDf.Tweet = twitterDf.Tweet.replace(r'#\w*', '', regex = True) # Erases hashtags
twitterDf.Tweet = twitterDf.Tweet.replace(r'https://.+', '', regex = True) # Erases URLS
twitterDf.Tweet = twitterDf.Tweet.replace(r'http://.+', '', regex = True) # Erases URLS
twitterDf.Tweet = twitterDf.Tweet.replace(r'=>', '', regex = True) # Erases =>
twitterDf.Tweet = twitterDf.dropna().Tweet.apply(lambda x: detect(str(x)) != 'en' and np.nan or x) # Creates NA place holder for text that is not in english 
twitterDf = twitterDf.dropna()


In [9]:
aggDf = twitterDf.drop(columns = 'Tweet')
aggDf.timeStamp = pd.to_datetime(aggDf.timeStamp) 
aggDf.set_index(aggDf["timeStamp"],inplace=True)
aggDf.resample('D').sum()


,Favourites,retweets,Compound,Positive,Neutral,Negative
timeStamp,,,,,,
2019-01-12,14,11,1.5504,0.836,3.938,0.227
2019-01-13,68,23,2.4158,0.726,7.134,0.140
2019-01-14,36,22,0.7916,0.407,3.487,0.106
